# Meteor Interceptor 

In [ ]:
using ModelingToolkit, DifferentialEquations, GLMakie
using ModelingToolkit: t_nounits as t, D_nounits as D

# 1. Meteor Interceptor Simulation - Proportional Navigation

# Parameters , N: Navigation constant, g: gravity
@parameters N g  

# Define variables
@variables xt(t) yt(t) vxt(t) vyt(t)
@variables xm(t) ym(t) vxm(t) vym(t)
@variables Δx(t) Δy(t) R2(t) Vc(t) λ(t) λ̇(t) aₙ(t) aₓ(t) aᵧ(t)

# Define the set of differential-algebraic equations
eqs = [
    D(xt) ~ vxt,
    D(yt) ~ vyt,
    
    D(vxt) ~ 0.0,        # Meteor assumed constant horizontal velocity
    D(vyt) ~ -g,         # Meteor vertical motion under gravity
    
    D(xm) ~ vxm,
    D(ym) ~ vym,
    
    Δx ~ xt - xm,
    Δy ~ yt - ym,
    R2  ~ Δx^2 + Δy^2,
    
    λ ~ atan(Δy, Δx),
    λ̇ ~ ((vyt - vym)*Δx - (vxt - vxm)*Δy) / R2,
    
    Vc ~ -((vxt - vxm)*Δx + (vyt - vym)*Δy) / sqrt(R2),
    
    aₙ ~ N * Vc * λ̇,
    
    aₓ ~ -aₙ * sin(λ),
    aᵧ ~  aₙ * cos(λ),
    
    D(vxm) ~ aₓ,
    D(vym) ~ aᵧ -g 
]

# Build the system
@mtkbuild odesys = ODESystem(eqs, t)

# Initial conditions and parameters
tstart = 0.0
tend = 50.0
tspan = (tstart, tend)
steps = 0.1
timesteps = tstart:steps:tend

# Initial states and parameters
u0 = [
    # Meteor
xt => 0.0     ,    # Directly above
yt => 30000.0  ,   # 30 km high

vxt => 400.0   ,   # Meteor moving rightwards
vyt => -600.0  ,   # Meteor falling down

# Interceptor
xm => -5000.0 ,   # Launch from 5 km west
ym => 0.0       ,  # On the ground

vxm => 800.0    ,  # Shooting toward meteor
vym => 1200.0     # Strong vertical component to climb

]

p = [
    N => 4.0, 
    g => 9.81   # Gravity m/s²
]

# Define the Callback (intercept within 50m)
function hit_condition(u, t, integrator)
    Δx = u[1] - u[5]
    Δy = u[2] - u[6]
    dist = sqrt(Δx^2 + Δy^2)
    return dist - 50.0
end

function hit_affect!(integrator)
    terminate!(integrator)
end

cb = ContinuousCallback(hit_condition, hit_affect!)

# Define the problem
odeprob = ODEProblem(odesys, u0, tspan, p)

# Solve
sol = solve(odeprob, Tsit5(); saveat=timesteps,dtmax=0.001 ,callback=cb)

# Setup figure and axis
fig = Figure(size=(800, 600))
ax = Axis(fig[1, 1], xlabel="X Position (m)", ylabel="Y Position (m)", title="Meteor Interception")

# Initial limits
limits!(ax, -6000, 6000, -1000, 11000)

# Scatter plots
meteor_plot = GLMakie.scatter!(ax, [sol[xt][1]], [sol[yt][1]], color=:orange, markersize=15, label="Meteor")
missile_plot = GLMakie.scatter!(ax, [sol[xm][1]], [sol[ym][1]], color=:red, markersize=15, label="Interceptor")

axislegend(ax)

# Create Observable index
frame_idx = Observable(1)

# Update function
on(frame_idx) do idx
    meteor_plot[1][] = [Point2f(sol[xt][idx], sol[yt][idx])]
    missile_plot[1][] = [Point2f(sol[xm][idx], sol[ym][idx])]
    
    # Adjust axis limits dynamically
    x_vals = [sol[xm][idx], sol[xt][idx]]
    y_vals = [sol[ym][idx], sol[yt][idx]]
    xmin, xmax = extrema(x_vals)
    ymin, ymax = extrema(y_vals)

    margin = 1000
    limits!(ax, xmin - margin, xmax + margin, ymin - margin, ymax + margin)
end

# Record animation
record(fig, "meteor_interceptor_simulation.mp4", 1:length(sol.t); framerate=15) do idx
    frame_idx[] = idx
end


Added a thrust parameter also 

In [ ]:
using ModelingToolkit, DifferentialEquations, GLMakie
using ModelingToolkit: t_nounits as t, D_nounits as D

# 2. Meteor Interceptor Simulation - Proportional Navigation with Thrust

# Parameters , N: Navigation constant, g: gravity, thrust: missile acceleration
@parameters N g thrust  

# Define variables
@variables xt(t) yt(t) vxt(t) vyt(t)
@variables xm(t) ym(t) vxm(t) vym(t)
@variables Δx(t) Δy(t) R2(t) Vc(t) λ(t) λ̇(t)
@variables v_mag(t) θ_dir(t) a_thrust_x(t) a_thrust_y(t)
@variables aₙ(t) aₓ(t) aᵧ(t)

# Define the set of differential-algebraic equations
eqs = [
    D(xt) ~ vxt,
    D(yt) ~ vyt,

    D(vxt) ~ 0.0,        # Meteor: constant horizontal velocity
    D(vyt) ~ -g,         # Meteor under gravity

    D(xm) ~ vxm,
    D(ym) ~ vym,

    Δx ~ xt - xm,
    Δy ~ yt - ym,
    R2 ~ Δx^2 + Δy^2,

    λ ~ atan(Δy, Δx),
    λ̇ ~ ((vyt - vym)*Δx - (vxt - vxm)*Δy) / R2,
    Vc ~ -((vxt - vxm)*Δx + (vyt - vym)*Δy) / sqrt(R2),

    v_mag ~ sqrt(vxm^2 + vym^2),
    θ_dir ~ atan(vym, vxm),

    a_thrust_x ~ thrust * cos(θ_dir),
    a_thrust_y ~ thrust * sin(θ_dir),

    aₙ ~ N * Vc * λ̇,
    aₓ ~ -aₙ * sin(λ) + a_thrust_x,
    aᵧ ~  aₙ * cos(λ) + a_thrust_y - g,

    D(vxm) ~ aₓ,
    D(vym) ~ aᵧ
]

# Build the system
@mtkbuild odesys = ODESystem(eqs, t)

# Initial time and simulation setup
tstart = 0.0
tend = 50.0
tspan = (tstart, tend)
steps = 0.1
timesteps = tstart:steps:tend

# Initial conditions and parameters
u0 = [
    # Meteor
    xt => 0.0,
    yt => 30000.0,
    vxt => 400.0,
    vyt => -600.0,

    # Interceptor
    xm => -5000.0,
    ym => 0.0,
    vxm => 800.0,
    vym => 1200.0
]

p = [
    N => 4.0,
    g => 9.81,
    thrust => 20.0  # Missile thrust acceleration (m/s²)
]

# Define the Callback for interception (within 50m)
function hit_condition(u, t, integrator)
    Δx = u[1] - u[5]
    Δy = u[2] - u[6]
    dist = sqrt(Δx^2 + Δy^2)
    return dist - 50.0
end

function hit_affect!(integrator)
    terminate!(integrator)
end

cb = ContinuousCallback(hit_condition, hit_affect!)

# Define and solve the ODE problem
odeprob = ODEProblem(odesys, u0, tspan, p)
sol = solve(odeprob, Tsit5(); saveat=timesteps, dtmax=0.001, callback=cb)

# Visualization
fig = Figure(size=(800, 600))
ax = Axis(fig[1, 1], xlabel="X Position (m)", ylabel="Y Position (m)", title="Meteor Interception with Thrust")
limits!(ax, -6000, 6000, -1000, 11000)

# Scatter plots
meteor_plot = GLMakie.scatter!(ax, [sol[xt][1]], [sol[yt][1]], color=:orange, markersize=15, label="Meteor")
missile_plot = GLMakie.scatter!(ax, [sol[xm][1]], [sol[ym][1]], color=:red, markersize=15, label="Interceptor")
axislegend(ax)

# Observable index
frame_idx = Observable(1)

# Update function for animation
on(frame_idx) do idx
    meteor_plot[1][] = [Point2f(sol[xt][idx], sol[yt][idx])]
    missile_plot[1][] = [Point2f(sol[xm][idx], sol[ym][idx])]

    x_vals = [sol[xm][idx], sol[xt][idx]]
    y_vals = [sol[ym][idx], sol[yt][idx]]
    xmin, xmax = extrema(x_vals)
    ymin, ymax = extrema(y_vals)

    margin = 1000
    limits!(ax, xmin - margin, xmax + margin, ymin - margin, ymax + margin)
end

# Record animation
record(fig, "meteor_interceptor_simulation_with_thrust.mp4", 1:length(sol.t); framerate=15) do idx
    frame_idx[] = idx
end
